In [1]:
import re
import os
import bcolz
%matplotlib inline
import numpy as np
import pandas as pd
from pprint import pprint
import _pickle as cPickle
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.utils.data_utils import get_file
from keras.layers import SimpleRNN, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.layers import Dense, Dropout, Activation, Flatten
from numpy.random import random, permutation, randn, normal, uniform, choice
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional

Using TensorFlow backend.


In [2]:
# path = get_file(fname = 'allPyPrograms_no_comments.txt', origin = 'file:///homw/wtemp/585/allPyPrograms_no_comments.txt')
path = os.getcwd() + '/allPyPrograms_no_comments.txt'
text_all = open(path).read()
text = text_all[:1000000]
print(len(text))

1000000


In [3]:
from pprint import pprint
pprint(text[:300])

('def sort_blocks():\n'
 "    with open('README.md', 'r') as read_me_file:\n"
 '        read_me = read_me_file.read()\n'
 "    table_of_contents = ''.join(read_me.split('- - -')[0])\n"
 "    blocks = ''.join(read_me.split('- - -')[1]).split('\\n# ')\n"
 '    for i in range(len(blocks)):\n'
 '        if i == 0:\n'
 '            blocks[i]')


In [4]:
words = re.findall(r"[\w']+", text)
print(len(words)+1)
words.append("")

108118


In [5]:
print(words[:30])

['def', 'sort_blocks', 'with', 'open', "'README", "md'", "'r'", 'as', 'read_me_file', 'read_me', 'read_me_file', 'read', 'table_of_contents', "''", 'join', 'read_me', 'split', "'", "'", '0', 'blocks', "''", 'join', 'read_me', 'split', "'", "'", '1', 'split', "'"]


In [6]:
char_to_index = {v:i for i,v in enumerate(words)}
index_to_char = {i:v for i,v in enumerate(words)}

In [7]:
total_index = [char_to_index[w] for w in words]

In [8]:
total_index[:10]

[108064, 192, 107351, 79735, 53640, 23669, 78978, 104042, 117, 125]

In [9]:
''.join(index_to_char[i] for i in total_index[:25])

"defsort_blockswithopen'READMEmd''r'asread_me_fileread_meread_me_filereadtable_of_contents''joinread_mesplit''0blocks''joinread_mesplit"

In [10]:
pred_num = 3
xin = [[total_index[j+i] for j in range(0, len(total_index)-1-pred_num, pred_num)] for i in range(pred_num)]
y = [total_index[i+pred_num] for i in range(0, len(total_index)-1-pred_num, pred_num)]

In [11]:
X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
Y = np.stack(y[:-2])

In [12]:
pprint(X)

[array([108064,  79735,  78978, ..., 108099, 108102, 108105]),
 array([   192,  53640, 104042, ..., 108100, 108103, 108106]),
 array([107351,  23669,    117, ..., 108101, 108104, 108107])]


In [13]:
Y[:8]

array([ 79735,  78978,    125,     90,    125, 108059, 105345,  99520])

In [14]:
X[0].shape, Y.shape

((36036,), (36036,))

In [15]:
hidden_layers = 256
vocab_size = len(words)
n_fac = 42

In [16]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 42)             4540956   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_1 (Dense)              (None, 108118)            27786326  
Total params: 32,403,826
Trainable params: 32,403,826
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [19]:
model.fit(np.stack(X, 1), Y, batch_size=64, epochs=20)

Epoch 1/20
36036/36036 [==============================] - 532s 15ms/step - loss: 8.3777
Epoch 2/20
36036/36036 [==============================] - 462s 13ms/step - loss: 7.1828
Epoch 3/20
36036/36036 [==============================] - 461s 13ms/step - loss: 5.7046
Epoch 5/20
36036/36036 [==============================] - 461s 13ms/step - loss: 4.8496
Epoch 6/20
36036/36036 [==============================] - 461s 13ms/step - loss: 3.9133
Epoch 7/20
36036/36036 [==============================] - 461s 13ms/step - loss: 2.9620
Epoch 8/20
36036/36036 [==============================] - 461s 13ms/step - loss: 2.2022
Epoch 9/20
36036/36036 [==============================] - 461s 13ms/step - loss: 1.7199
Epoch 10/20
36036/36036 [==============================] - 462s 13ms/step - loss: 1.4154
Epoch 11/20
36036/36036 [==============================] - 465s 13ms/step - loss: 1.1917
Epoch 12/20
36036/36036 [==============================] - 462s 13ms/step - loss: 1.0361
Epoch 13/20
36036/36036 [====

In [20]:
model.save_weights('simpleRNN_3predword.h5')

In [21]:
model.load_weights('simpleRNN_3predword.h5')

In [22]:
model.save_weights('simpleRNN_7predword.h5')

In [23]:
model.load_weights('simpleRNN_7predword.h5')

In [28]:
def predict_next_char(inp):
    inp = re.findall(r"[\w']+", inp)
    for i in range(len(inp)):
        if inp[i] not in char_to_index.keys():
            inp[i] = ""
    l = len(inp)
    if l > pred_num:
        inp = inp[-pred_num:]
    if l < pred_num:
        for i in range(pred_num-l):
            inp.insert(0, "")
#     print(len(inp))
    index = [char_to_index[i] for i in inp]
    arr = np.expand_dims(np.array(index), axis=0)
    prediction = model.predict(arr)
    return index_to_char[np.argmax(prediction)]

In [29]:
predict_next_char('def main')

'self'

In [30]:
predict_next_char('l = len( arr ) arr.insert')

'0'

In [27]:
prog = "while True:"
for i in range(100):
    prog = prog +" "+ predict_next_char(prog[-pred_num:])
prog

'while True: param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param param'

In [31]:
def recom_model1(observed_seq):
    # Get the last letter
    last_item = observed_seq[-1]
    word = ""
    thresh = 100
    while thresh > 0 and (len(word) < 1 or word[-1] != " "):
        thresh -= 1
        if len(word) > 100:
            return word
        seen = observed_seq + word
        p = predict_next_char(seen)
        word = word + p
        last_item = p
    return word

In [32]:
import sys,os
import subprocess
import evalerr
import sys


# In[12]:


TEST_DIR = '/home/wtemp/585/error/Test_Dataset/'

HOLE = "PREDICTION_HOLE_PLACEHOLDER"

PRED_DIR = "/home/wtemp/585/error/Predicted_Results/"
result_dict={}

evalerr.test(recom_model1)

dummy.py  has no hole to predict
(b'', b'')


(70, 16, 1)

In [ ]:
ys = [[total_index[j+i] for j in range(1, len(total_index)-pred_num, pred_num)] for i in range(pred_num)]

In [ ]:
Y_return = [np.stack(ys[i][:-2]) for i in range(pred_num)]

In [ ]:
X

In [ ]:
Y_return

In [ ]:
vocab_size = 97
n_fac = 42
hidden_layers = 256

In [ ]:
return_model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, return_sequences=True, activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

In [ ]:
return_model.summary()

In [ ]:
return_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
X_model = np.stack(X, 1)
Y_model = np.expand_dims(np.stack(Y_return, 1), axis=-1)

In [ ]:
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.save_weights('return_sequences_25.h5')

In [ ]:
def predict_every_char(inp):
    l = []
    p = 0
    while p<len(inp):
        pre_inp = inp[p:p+pred_num]
        if len(pre_inp) < pred_num:
            pre_inp = pre_inp + ' '*(pred_num - len(pre_inp))
            l.append(pre_inp)
        else:
            l.append(pre_inp) 
        p+=pred_num

#     index = [char_to_index[i] for i in inp]
#     arr = np.expand_dims(index, axis=0)
#     prediction = return_model.predict(arr)
#     return ''.join([index_to_char[np.argmax(i)] for i in prediction[0]])
    
    final = []
    for half in l:
        index = [char_to_index[i] for i in half]
        arr = np.expand_dims(index, axis=0)
        prediction = return_model.predict(arr)
        final.append(''.join([index_to_char[np.argmax(i)] for i in prediction[0]]))
    
    return ''.join(final)

In [ ]:
predict_every_char('qor item i')

In [ ]:
def recom_model(observed_seq):
    # Get the last letter
    last_item = observed_seq[-1]
    word = ""
    thresh = 100
    while thresh > 0 and (len(word) < 1 or word[-1] != " "):
        thresh -= 1
        if len(word) > 100:
            return word
        p = predict_every_char(last_item)
        word = word + p
        last_item = p
    return word

In [ ]:
import sys,os
import subprocess
import evalerr
import sys


# In[12]:


TEST_DIR = '/home/wtemp/585/error/Test_Dataset/'

HOLE = "PREDICTION_HOLE_PLACEHOLDER"

PRED_DIR = "/home/wtemp/585/error/Predicted_Results/"
result_dict={}

evalerr.test(recom_model)